## submit jobs to cluster

In [230]:
import os
from rosemary import jpt_in_notebook, jpt_setup; jpt_setup()
from llm.submit import submit_job, multiline_to_singleline, shell_scripts_template_slurm

first_N = None
log_dir = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/'

test_run = 1
test_run = bool(test_run)

####
model_name = 'all-mpnet-base-v2'; encode_fn_type = 'input'; normalize_embeddings = True

datasets = 'wizardlm'
## 
clustering_fn_list = []
clustering_fn_list += [
    f'cl=kmeans_nc={n_clusters}' for n_clusters in [100, 300, 1000]
]

# first_N = 10_000
# clustering_fn = f'cl=kmeansminibatch_nc={n_clusters}'
# clustering_fn = f'cl=kmeansminibatch_nc={n_clusters}_bsz=256'


###

cmds = []
for clustering_fn in clustering_fn_list:
    save_dir = (f"/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/"
                f"clustering/{encode_fn_type+'/' if encode_fn_type!='sft' else ''}{model_name}/"
                f"{dataset+'(N='+str(first_N)+')' if first_N else dataset}/{clustering_fn}")

    cmd = f"""
    python note_pruning_clustering.py \
        --model_name {model_name} \
        --dataset {dataset} \
        --encode_fn_type {encode_fn_type} \
        --clustering_fn {clustering_fn} \
        {'--normalize_embeddings' if normalize_embeddings else ''} \
        {'--first_N '+str(first_N) if first_N else ''} \
        --save_dir {save_dir} \
    """.strip()
    cmd = multiline_to_singleline(cmd)

    shell_scripts = shell_scripts_template_slurm.format(
        conda_env='open-instruct',
        cwd=os.getcwd(),
        cmd=cmd,
        log_dir=log_dir,
        save_dir=save_dir)
    out = submit_job(
        shell_scripts, 
        job_name=f'cluster.{dataset}.{clustering_fn}', 
        nodes=1,
        num_cpus=64,
        cpu_mem=128,
        num_gpus=1,
        gpu_type='v100',
        test_run=test_run,
        job_duration=6,
    )
    print(cmd)
    cmds.append(cmd)
    
print(f'#cmds: {len(cmds)}')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



Submiting job with:
{
    "job_name": "cluster.wizardlm.cl=kmeans_nc=100",
    "nodes": 1,
    "num_cpus": 64,
    "cpu_mem": 128,
    "num_gpus": 1,
    "gpu_type": "v100",
    "test_run": false,
    "queue": "el8",
    "num_jobs": 1
}
python note_pruning_clustering.py --model_name all-mpnet-base-v2 --dataset wizardlm --encode_fn_type input --clustering_fn cl=kmeans_nc=100 --normalize_embeddings --save_dir /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/clustering/input/all-mpnet-base-v2/wizardlm/cl=kmeans_nc=100

Submiting job with:
{
    "job_name": "cluster.wizardlm.cl=kmeans_nc=300",
    "nodes": 1,
    "num_cpus": 64,
    "cpu_mem": 128,
    "num_gpus": 1,
    "gpu_type": "v100",
    "test_run": false,
    "queue": "el8",
    "num_jobs": 1
}
python note_pruning_clustering.py --model_name all-mpnet-base-v2 --dataset wizardlm --encode_fn_type input --clustering_fn cl=kmeans_nc=300 --normalize_embeddings --save_dir /gpfs/u/home/PTFM/PTFMqngp/scra

## analyze clustering results

In [ ]:



    save_dir = (f"/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/scripts/"
                f"clustering/{encode_fn_type+'/' if encode_fn_type!='sft' else ''}{model_name}/"
                f"{dataset+'(N='+str(first_N)+')' if first_N else dataset}/{clustering_fn}")


## clustering code base


In [1]:
from rosemary import jpt_parse_args, jpt_setup, jpt_in_notebook; jpt_setup()

if jpt_in_notebook():
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['CUDA_VISIBLE_DEVICES'].split(',')[0] 
    # '0,1,2,3,4,5'
    print(os.environ['CUDA_VISIBLE_DEVICES'])
# !nvidia-smi

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


4


In [211]:
import re
import argparse
import json
import pickle
import string
import time
import matplotlib.pyplot as plt

import numpy as np
import scipy
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sklearn.cluster import KMeans, AgglomerativeClustering
import torch
from tqdm import tqdm
import pandas as pd

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel

from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F

import time
from note_pruning_analysis import (
    get_lm_output,
    get_dataset,
)
from note_pruning_clustering import (
    pairwise_cosine_distance,
    clustering_sort_by_cluster_size,
    clustering_algorithm_scores,
    clustering_dist_to_centroids,
    clustering_knn_withincluster,
    clustering_run,
    clustering_compute_and_save_results,
)


In [20]:
model_name = 'all-mpnet-base-v2'

dataset = 'wizardlm'
encode_fn_type = 'input'
first_N = 10_000

n_clusters = 50
clustering_fn = f'cl=kmeans_nc={n_clusters}'
# clustering_fn = f'cl=kmeansminibatch_nc={n_clusters}'
# clustering_fn = f'cl=kmeansminibatch_nc={n_clusters}_bsz=256'

In [21]:
save_dir = os.path.join('clustering', encode_fn_type, model_name, dataset, clustering_fn)
os.makedirs(save_dir, exist_ok=True)
save_dir

'clustering/input/all-mpnet-base-v2/wizardlm/cl=kmeans_nc=50'

In [5]:

ds = get_dataset(dataset, processed=True)
if encode_fn_type == 'input':
    def get_user_prompt_fn(example):
        example['text'] = example['messages'][0]['content']
        return example
    ds = ds.map(get_user_prompt_fn, num_proc=16)

ds

Found cached dataset json (/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/processed/wizardlm/json/default-a3f71ae376e517fb/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
Loading cached processed dataset at /gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/processed/wizardlm/json/default-a3f71ae376e517fb/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-b089bdb80a735e3f_*_of_00016.arrow


Dataset({
    features: ['dataset', 'id', 'messages', 'text'],
    num_rows: 143000
})

In [151]:
d = get_lm_output(dataset, 
                  model_name, 
                  encode_fn_type=encode_fn_type,
                  return_text_embedding=True,)
X = d['text_embedding']
X = X / np.linalg.norm(X, axis=-1, keepdims=True)

if first_N:
    ds = ds.select(range(first_N))
    X = X[:first_N]

In [207]:

info = {}
info['N'] = len(X)
info['dataset'] = 'dataset'
info['model_name'] = model_name
info['encode_fn_type'] = encode_fn_type

t0 = time.time()
Y, C, clustering_model = clustering_run(clustering_fn, X)
info['time_elapsed'] = time.time()-t0
info['scores'] = {}
info['scores'].update({'inertia': clustering_model.inertia_})
info['scores'].update(clustering_algorithm_scores(X, Y))
info['cluster_sizes'] = np.unique(Y, return_counts=True)[1].tolist()

with open(os.path.join(save_dir, 'info.json'), 'w') as f:
    json.dump(info, f, ensure_ascii=False, indent=4)

clustering_compute_and_save_results(X, Y, C, ds=ds, save_dir=save_dir)


Initialization complete
Iteration 0, inertia 12931.5439453125.
Iteration 1, inertia 7934.09423828125.
Iteration 2, inertia 7803.97412109375.
Iteration 3, inertia 7746.900390625.
Iteration 4, inertia 7716.85693359375.
Iteration 5, inertia 7699.97998046875.
Iteration 6, inertia 7687.66845703125.
Iteration 7, inertia 7676.87548828125.
Iteration 8, inertia 7665.97412109375.
Iteration 9, inertia 7656.21435546875.
Iteration 10, inertia 7647.6005859375.
Iteration 11, inertia 7639.76513671875.
Iteration 12, inertia 7632.611328125.
Iteration 13, inertia 7625.7197265625.
Iteration 14, inertia 7619.814453125.
Iteration 15, inertia 7615.34326171875.
Iteration 16, inertia 7611.671875.
Iteration 17, inertia 7608.85400390625.
Iteration 18, inertia 7606.2548828125.
Iteration 19, inertia 7604.70654296875.
Iteration 20, inertia 7603.83349609375.
Iteration 21, inertia 7603.06787109375.
Iteration 22, inertia 7602.5693359375.
Iteration 23, inertia 7602.13232421875.
Iteration 24, inertia 7601.8115234375.
It

/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
